# Postgres Data Exploration
Use the helper functions below to inspect tables in the project database.


**Setup**
- Ensure `.env` contains a valid `DATABASE_URL` pointing at Postgres.
- Update `table_to_preview` or the custom SQL cells to target the tables you want to inspect.


In [ ]:
from __future__ import annotations

import os
from pathlib import Path
from typing import Any

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import MetaData, Table, create_engine, select, text
from sqlalchemy.engine import Engine

PROJECT_ROOT = Path("../../../").resolve()
load_dotenv(PROJECT_ROOT / "backend/.env")

DATABASE_URL = os.environ["DATABASE_URL"]
engine: Engine = create_engine(DATABASE_URL, future=True)
metadata = MetaData()

print(f"Engine ready for {engine.url.render_as_string(hide_password=True)}")


KeyError: 'DATABASE_URL'

In [ ]:
def run_query(sql: str, params: dict[str, Any] | None = None) -> pd.DataFrame:
    """Execute a raw SQL query and return a DataFrame."""
    query = text(sql)
    with engine.connect() as connection:
        return pd.read_sql(query, connection, params=params)


def preview_table(table_identifier: str, limit: int = 10) -> pd.DataFrame:
    """Load a table by name (optionally schema-qualified) for a quick look."""
    schema: str | None = None
    table_name = table_identifier
    if "." in table_identifier:
        schema, table_name = table_identifier.split(".", 1)

    table_obj = Table(table_name, metadata, autoload_with=engine, schema=schema)
    statement = select(table_obj).limit(limit)
    with engine.connect() as connection:
        return pd.read_sql(statement, connection)


In [ ]:
pd.options.display.max_rows = 20
pd.options.display.max_columns = None
pd.options.display.width = 0


In [5]:
tables = run_query(
    """
    SELECT table_schema, table_name
    FROM information_schema.tables
    WHERE table_type = 'BASE TABLE'
      AND table_schema NOT IN ('pg_catalog', 'information_schema')
    ORDER BY table_schema, table_name
    """
)
tables


NameError: name 'run_query' is not defined

In [ ]:
# Replace with the table you want to preview, e.g. "public.players" or "public.games"
table_to_preview = "public.players"
preview_table(table_to_preview, limit=25)


In [ ]:
# Customize the SQL below for deeper exploration.
custom_sql = """
-- Example: row counts per table
SELECT table_schema,
       table_name,
       pg_total_relation_size(format('%s.%s', table_schema, table_name)) AS bytes
FROM information_schema.tables
WHERE table_type = 'BASE TABLE'
  AND table_schema NOT IN ('pg_catalog', 'information_schema')
ORDER BY bytes DESC
LIMIT 20
"""

run_query(custom_sql)


## Quarterback Metrics Table
Aggregate weekly quarterback stats into season-level metrics including average depth of target (ADOT) and rushing usage per game.


In [ ]:
QB_METRICS_SQL = """
SELECT
    player_id,
    player_display_name,
    season,
    team,
    COUNT(*) AS games_played,
    SUM(attempts) AS pass_attempts,
    SUM(completions) AS completions,
    SUM(passing_air_yards) AS passing_air_yards,
    SUM(passing_yards) AS passing_yards,
    SUM(carries) AS rush_attempts,
    SUM(rushing_yards) AS rushing_yards,
    SUM(rushing_tds) AS rushing_tds
FROM public.nfl_weekly_stats
WHERE position = 'QB'
  AND season BETWEEN :season_start AND :season_end
  AND season_type = :season_type
GROUP BY player_id, player_display_name, season, team
HAVING SUM(attempts) >= :min_attempts
   AND COUNT(*) >= :min_games
ORDER BY season DESC, SUM(attempts) DESC
"""

def load_qb_metrics(
    *,
    season_start: int,
    season_end: int,
    min_attempts: int = 150,
    min_games: int = 6,
    season_type: str = "REG",
) -> pd.DataFrame:
    """Return aggregated quarterback metrics for the requested seasons."""
    params = {
        "season_start": season_start,
        "season_end": season_end,
        "min_attempts": min_attempts,
        "min_games": min_games,
        "season_type": season_type,
    }
    frame = run_query(QB_METRICS_SQL, params=params)
    if frame.empty:
        return frame

    metrics = frame.copy()
    metrics["adot"] = metrics["passing_air_yards"].div(
        metrics["pass_attempts"].replace({0: pd.NA})
    )
    metrics["ypa"] = metrics["passing_yards"].div(
        metrics["pass_attempts"].replace({0: pd.NA})
    )
    safe_games = metrics["games_played"].replace({0: pd.NA})
    metrics["pass_attempts_per_game"] = metrics["pass_attempts"].div(safe_games)
    metrics["rush_att_per_game"] = metrics["rush_attempts"].div(safe_games)
    metrics["rush_yds_per_game"] = metrics["rushing_yards"].div(safe_games)

    metric_columns = [
        "adot",
        "ypa",
        "pass_attempts_per_game",
        "rush_att_per_game",
        "rush_yds_per_game",
    ]
    metrics[metric_columns] = metrics[metric_columns].apply(
        pd.to_numeric, errors="coerce"
    )
    return metrics


In [ ]:
season_type_default = "REG"
season_window = run_query(
    """
    SELECT MIN(season) AS min_season, MAX(season) AS max_season
    FROM public.nfl_weekly_stats
    WHERE position = 'QB'
      AND season_type = :season_type
    """,
    params={"season_type": season_type_default},
)

qb_min_season: int | None
qb_max_season: int | None

if season_window.empty or season_window.loc[0, "min_season"] is None:
    qb_min_season = None
    qb_max_season = None
    print(
        "No quarterback data detected for season type '",
        season_type_default,
        "'. Update qb_query_options manually.",
        sep="",
    )
else:
    qb_min_season = int(season_window.loc[0, "min_season"])
    qb_max_season = int(season_window.loc[0, "max_season"])
    print(f"Quarterback seasons available: {qb_min_season}–{qb_max_season}")

if qb_min_season is None:
    qb_default_start = None
    qb_default_end = None
else:
    lookback_span = 2  # last three seasons by default when available
    qb_default_end = qb_max_season
    qb_default_start = max(qb_min_season, qb_max_season - lookback_span)


In [ ]:
qb_query_options = {
    "season_start": qb_default_start or 2025,
    "season_end": qb_default_end or (qb_default_start or 2025),
    "min_attempts": 100,
    "min_games": 4,
    "season_type": season_type_default,
}

print(
    "Using QB filters:",
    qb_query_options,
)

qb_metrics = load_qb_metrics(**qb_query_options)

if qb_metrics.empty:
    print(
        "No quarterback rows found. Adjust season_start/season_end or lower the"
        " min_attempts/min_games thresholds in qb_query_options."
    )
else:
    display_columns = [
        "player_display_name",
        "season",
        "team",
        "games_played",
        "pass_attempts",
        "pass_attempts_per_game",
        "passing_air_yards",
        "adot",
        "ypa",
        "rush_attempts",
        "rush_att_per_game",
        "rush_yds_per_game",
    ]
    available_display_columns = [
        column for column in display_columns if column in qb_metrics.columns
    ]
    qb_view = qb_metrics.loc[:, available_display_columns].copy()
    metric_display_columns = [
        column
        for column in [
            "adot",
            "ypa",
            "pass_attempts_per_game",
            "rush_att_per_game",
            "rush_yds_per_game",
        ]
        if column in qb_view.columns
    ]
    if metric_display_columns:
        qb_view[metric_display_columns] = qb_view[metric_display_columns].round(2)
    qb_view.sort_values(
        ["season", "adot"],
        ascending=[False, False],
        inplace=True,
    )
    qb_view.head(20)
